In [ ]:
EC2_IP="ec2-52-66-150-205.ap-south-1.compute.amazonaws.com"

In [ ]:
import boto3
import os

# Initialize the Rekognition client
rekognition = boto3.client('rekognition')

# Create a collection
def create_collection(collection_id):
    try:
/home/ubuntu/Swift/backend        response = rekognition.create_collection(CollectionId=collection_id)
        print(f"Collection {collection_id} created. ARN: {response['CollectionArn']}")
    except rekognition.exceptions.ResourceAlreadyExistsException:
        print(f"Collection {collection_id} already exists.")

# Index a face
def index_face(collection_id, image_path, employee_id):
    with open(image_path, 'rb') as image:
        response = rekognition.index_faces(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            ExternalImageId=employee_id,
            DetectionAttributes=['ALL']
        )
    print(f"Face indexed for employee {employee_id}")
    return response['FaceRecords'][0]['Face']['FaceId']

# Recognize a face
def recognize_face(collection_id, image_path):
    with open(image_path, 'rb') as image:
        response = rekognition.search_faces_by_image(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            MaxFaces=1,
            FaceMatchThreshold=95
        )
    
    if response['FaceMatches']:
        return response['FaceMatches'][0]['Face']['ExternalImageId']
    else:
        return None


In [3]:
import boto3
import os

def create_collection(collection_id):
    try:
        response = rekognition.create_collection(CollectionId=collection_id)
        print(f"Collection {collection_id} created. ARN: {response['CollectionArn']}")
    except rekognition.exceptions.ResourceAlreadyExistsException:
        print(f"Collection {collection_id} already exists.")

# Initialize the Rekognition client
rekognition = boto3.client('rekognition')
collection_id = "StudentFaces"
create_collection(collection_id)

Collection StudentFaces created. ARN: aws:rekognition:ap-south-1:961341519735:collection/StudentFaces


In [ ]:
student_folder = "student_photos"
for filename in os.listdir(student_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        employee_id = os.path.splitext(filename)[0]
        image_path = os.path.join(student_folder, filename)
        index_face(collection_id, image_path, employee_id)

In [ ]:


    # Index faces (add new employees)
    employee_folder = "student_photos"
    for filename in os.listdir(employee_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            employee_id = os.path.splitext(filename)[0]
            image_path = os.path.join(employee_folder, filename)
            index_face(collection_id, image_path, employee_id)

    # Recognize face (mark attendance)
    attendance_photo = "attendance_photo.jpg"
    recognized_employee = recognize_face(collection_id, attendance_photo)
    
    if recognized_employee:
        print(f"Attendance marked for employee: {recognized_employee}")
    else:
        print("No matching face found.")

In [ ]:
import boto3
import os

def create_topic(sns_client, topic_name):
    """
    Create an SNS topic if it doesn't exist
    
    Args:
        sns_client: Boto3 SNS client
        topic_name: Name of the topic to create
    
    Returns:
        topic_arn: ARN of the created/existing topic
    """
    try:
        response = sns_client.create_topic(Name=topic_name)
        return response['TopicArn']
    except Exception as e:
        print(f"Error creating topic: {e}")
        raise

def subscribe_email(sns_client, topic_arn, email):
    """
    Subscribe an email address to the SNS topic
    
    Args:
        sns_client: Boto3 SNS client
        topic_arn: ARN of the topic
        email: Email address to subscribe
    
    Returns:
        subscription_arn: ARN of the subscription
    """
    try:
        response = sns_client.subscribe(
            TopicArn=topic_arn,
            Protocol='email',
            Endpoint=email
        )
        return response['SubscriptionArn']
    except Exception as e:
        print(f"Error subscribing email: {e}")
        raise

def publish_message(sns_client, topic_arn, subject, message):
    """
    Publish a message to the SNS topic
    
    Args:
        sns_client: Boto3 SNS client
        topic_arn: ARN of the topic
        subject: Email subject
        message: Email message body
    """
    try:
        response = sns_client.publish(
            TopicArn=topic_arn,
            Subject=subject,
            Message=message
        )
        print(f"Message published. MessageId: {response['MessageId']}")
    except Exception as e:
        print(f"Error publishing message: {e}")
        raise

def main():
    # # Initialize the SNS client
    # sns_client = boto3.client('sns',
    #     aws_access_key_id=os.getenv("ACCESS_KEY"),
    #     aws_secret_access_key=os.getenv("SECRET_KEY"),
    #     region_name='ap-south-1',
    # )
    
    # Create a topic or use existing one
    topic_name = 'TestTopic'
    topic_arn = create_topic(sns_client, topic_name)
    
    # List of email recipients
    email_recipients = [
        'praveentn1234@gmail.com',
        'praveenkumar.e2022@vitstudent.ac.in'
    ]
    
    # # Subscribe each email to the topic
    for email in email_recipients:
        subscription_arn = subscribe_email(sns_client, topic_arn, email)
        print(f"Subscribed {email} to topic")
    
    # Publish a message
    subject = "Test Email Subject"
    message = "This is a test message sent via AWS SNS"
    publish_message(sns_client, topic_arn, subject, message)

if __name__ == "__main__":
    main()

Subscribed praveentn1234@gmail.com to topic
Subscribed praveenkumar.e2022@vitstudent.ac.in to topic
Message published. MessageId: 8effc3b9-a1cb-5e52-bb01-9c1265230c12
